### vLLM trial --FAILED


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
import torch

model_id = "amazon/MistralLite"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             torch_dtype=torch.float16,
                                             use_flash_attention_2=True,
                                             device_map="auto",)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)
prompt = "<|prompter|>What are the main challenges to support a long context for LLM?</s><|assistant|>"

sequences = pipeline(
    prompt,
    max_new_tokens=400,
    do_sample=False,
    return_full_text=False,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"{seq['generated_text']}")

ValueError: Tokenizer class LlamaTokenizer does not exist or is not currently imported.

## GPT 3.5 16k trial

In [ ]:
from openai import OpenAI
client = OpenAI(api_key = '<insert API key here>' )        # key removed for privacy, use appropriate key here
import time

In [ ]:
import os
import glob
import pandas as pd

directory_path = 'Patients_Datewise_Texts_With_First_Occurance'

csv_files = glob.glob(os.path.join(directory_path, '*.csv'))
dfs = []
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    dfs.append(df)

In [ ]:
df = dfs[2]

,CHARTDATE,TEXT,FIRST_OCCURANCE
0,2123-08-23,[**2123-8-23**] 12:30 PM\n CHEST (PORTABLE AP)...,0
1,2123-08-24,Smicu nsg admission note\n65 women recently dx...,0
2,2123-08-25,[**2123-8-25**] 9:02 PM\n CHEST (PORTABLE AP) ...,0
3,2123-08-26,[**2123-8-26**] 2:17 PM\n MR HEAD W & W/O CONT...,0


In [ ]:
df

,CHARTDATE,TEXT,FIRST_OCCURANCE
0,2123-08-23,[**2123-8-23**] 12:30 PM\n CHEST (PORTABLE AP)...,0
1,2123-08-24,Smicu nsg admission note\n65 women recently dx...,0
2,2123-08-25,[**2123-8-25**] 9:02 PM\n CHEST (PORTABLE AP) ...,0
3,2123-08-26,[**2123-8-26**] 2:17 PM\n MR HEAD W & W/O CONT...,0
4,2123-08-27,[**2123-8-27**] 12:51 AM\n CHEST (PORTABLE AP)...,0
5,2123-08-29,[**2123-8-29**] 2:32 PM\n CHEST (PA & LAT); CH...,0
6,2123-08-30,[**2123-8-30**] 6:18 PM\n CHEST (PORTABLE AP) ...,0
7,2123-08-31,[**2123-8-31**] 10:57 AM\n CHEST (PORTABLE AP)...,0
8,2123-09-01,[**2123-9-1**] 7:09 AM\n CHEST (PORTABLE AP) ...,0
9,2123-09-02,[**2123-9-2**] 8:33 AM\n CHEST (PORTABLE AP) ...,0


In [ ]:
import re
import os
import glob
import pandas as pd
from tqdm import tqdm

In [ ]:
directory_path = 'Patients_Datewise_Texts_With_First_Occurance'

csv_files = glob.glob(os.path.join(directory_path, '*.csv'))
dfs = []

already_saved = glob.glob(os.path.join('patients_datewise_gpt35', '*.csv'))

for csv_file in tqdm(csv_files):
    out_filename = 'gpt35_16k_' + csv_file.split('\\')[-1]
    if os.path.join('patients_datewise_gpt35', out_filename) in already_saved:
        continue
    
    df = pd.read_csv(csv_file)
    res, summaries, answers = [], [], []
    summary = ''
    token_len_exceeded_error = False
    for i, row in df.iterrows():
        try:
            text = 'Prior patient reports summary: ' + summary + '\nCurrent report:\n' + row['TEXT']
            start = time.time()
            completion = client.chat.completions.create(
                model="gpt-3.5-turbo-1106",
                messages=[
                    {"role": "system", "content": "You are an assistant for a medical practitioner, skilled in analyzing and summarizing medical notes and reports."},
                    {"role": "user", "content": text + ".\n Using the above medical notes, please give your diagnosis and also determine if the patient has pulmonary embolism. Summarize the report (start with Summary:), give your short diagnosis (start with Diagnosis:), and answer the pulmonary embolism question only with a yes or no. If pulmonary embolism cannot be established, please say no and nothing more."},
                ]
            )
            gentime = time.time() - start
            ans = completion.choices[0].message.content
            pattern = re.compile(r'Summary:(.*?)(?:Pulmonary embolism:)(.*)', re.DOTALL | re.IGNORECASE)
            summary_match = pattern.search(ans)
            summary = summary_match.group(1).strip() if summary_match else summary
            summaries.append(summary)

            # pe_match = re.search(r'Pulmonary Embolism: (.*)', ans, re.IGNORECASE)
            pe_text = summary_match.group(2).strip() if summary_match else ""
            res.append(pe_text)
            answers.append(ans)
            # print(ans, '\n', gentime)
        except Exception as e:
            print(e)
            token_len_exceeded_error = True
            break

    if token_len_exceeded_error:
        continue

    try:
        assert len(res) == len(df) == len(summaries) == len(answers), 'Lengths mismatch'
    except AssertionError:
        continue
    
    df['Summary'] = summaries
    df['PE_Result_GPT35_16k'] = res
    df['Answers'] = answers
    
    df.to_csv('patients_datewise_gpt35/' + out_filename, index=False)

In [ ]:
# df.drop(columns=['Summary', 'PE_Result_GPT35_16k', 'Answers'], inplace=True)
# df.head(2)

In [ ]:
# start = time.time()
# completion = client.chat.completions.create(
#   model="gpt-3.5-turbo",
#   messages=[
#     {"role": "system", "content": "You are an assistant for a medical practitioner, skilled in analyzing and summarizing medical notes and reports."},
#     {"role": "user", "content": "f{} Using the above medical notes, please give your diagnosis and also determine if the patient has pulmonary embolism. Keep your diagnosis short, and answer the pulmonary embolism question with a yes or no."},
#   ]
# )
# gentime = time.time() - start
# print(completion.choices[0].message, '\n', gentime)

ChatCompletionMessage(content="In the realm of code, where logic pervades,\nI'll weave a tale of a looping crusade.\nA concept profound, yet elegantly clear,\nRecursion, dear friends, is what we shall hear.\n\nImagine a function, a humble charmer,\nWhose power lies not in its initial armor.\nA call, it makes, to itself with grace,\nLike mirrors reflecting, within time and space.\n\nA seed of thought, a problem to unravel,\nEach iteration, a layer to travel.\nWith each recursion, it breaks it down,\nInto smaller fragments, truth shall be found.\n\nA fractal of code, a path untold,\nExpanding dimensions, behold, behold!\nIt dances with grace, in a mystical trance,\nUnraveling the puzzle, giving it a chance.\n\nLike an echo resounding, back and forth,\nIt explores the labyrinth, south, east, and north.\nWith keen observation, it scrutinizes,\nUnraveling mysteries, solving the prizes.\n\nBut beware, my friends, for recursion can be sly,\nCaught in a loop, it may make you cry.\nAn end condi